In [1]:
!pip install flask flask-cors transformers pyngrok
!huggingface-cli login

from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
from transformers import pipeline
from pyngrok import ngrok
import time

app = Flask(__name__)
CORS(app)

# Initialize translation pipeline
TRANSLATION_MODEL = "facebook/m2m100_418M"
translator = pipeline("translation", model=TRANSLATION_MODEL)

SUPPORTED_LANGUAGES = {
    'en': 'English',
    'hi': 'Hindi',
    'ta': 'Tamil',
    'te': 'Telugu',
    'ml': 'Malayalam',
    'fr': 'French',
    'es': 'Spanish',
    'de': 'German'
}

chat_history = []
current_language = 'en'

# Basic rule-based chatbot responses
RESPONSE_MAP = {
    "hello": "Hi! How can I help you?",
    "hi": "Hello! What's up?",
    "how are you?": "I'm fine, thank you! How about you?",
    "what is your name?": "I'm your chatbot assistant!",
    "bye": "Goodbye! Have a nice day!",
    "thank you": "You're welcome!"
}

def chatbot_response(message):
    return RESPONSE_MAP.get(message.lower(), "I'm not sure how to respond to that.")

def translate_text(text, target_lang):
    if target_lang == 'en' or not text:
        return text
    try:
        return translator(text, src_lang="en", tgt_lang=target_lang, max_length=400)[0]['translation_text']
    except Exception as e:
        print(f"Translation error: {e}")
        return text

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE, chat_history=chat_history)

@app.route('/send_message', methods=['POST'])
def send_message():
    data = request.json
    user_message = data['message']
    timestamp = time.strftime("%H:%M")

    # Store user message
    chat_history.append({
        'type': 'user',
        'text': user_message,
        'time': timestamp,
        'original': user_message
    })

    # Generate chatbot response
    bot_response = chatbot_response(user_message)
    translated_response = translate_text(bot_response, current_language)
    chat_history.append({
        'type': 'bot',
        'text': translated_response,
        'time': timestamp,
        'original': bot_response
    })

    return jsonify({'chat_history': chat_history})

@app.route('/translate_chat', methods=['POST'])
def translate_chat():
    global current_language
    current_language = request.json['language']

    # Retranslate all bot messages
    for msg in chat_history:
        if msg['type'] == 'bot':
            msg['text'] = translate_text(msg['original'], current_language)

    return jsonify({'chat_history': chat_history})

@app.route('/get_chat')
def get_chat():
    return jsonify({'chat_history': chat_history})

if __name__ == '__main__':
    ngrok.set_auth_token("2sM1gKkJENb8i0cbgj4s1P7hX4O_35m1hEdUiZeKNwjgeFNv8")
    public_url = ngrok.connect(5000).public_url
    print(f" * Public URL: {public_url}")
    app.run(host='0.0.0.0', port=5000)


KeyboardInterrupt: 